# Embeddings
* Transform the small parts of text in numbers (vectors) that are easily stored and searched by vector databases.

In [ ]:
#pip install python-dotenv

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [3]:
#!pip install langchain

## Connect with an LLM

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [4]:
#!pip install langchain-openai

* NOTE: Since right now is the best LLM in the market, we will use OpenAI by default. You will see how to connect with other Open Source LLMs like Llama3 or Mistral in a next lesson.

In [2]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-3.5-turbo-0125")

## Reminder: Steps of the RAG process.
* When you load a document, you end up with strings. Sometimes the strings will be too large to fit into the context window. In those occassions we will use the RAG technique:
    * Split document in small chunks.
    * **Transform text chunks in numeric chunks (embeddings)**.
    * Load embeddings to a vector database (aka vector store).
    * Load question and retrieve the most relevant embeddings to respond it.
    * Sent the embeddings to the LLM to format the response properly.

## Splitters: divide the loaded document in small chunks of text
* Also called "Document Transformers".
* See the documentation page [here](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/).
* See the list of built-in splitters [here](https://python.langchain.com/v0.1/docs/integrations/document_transformers/).

#### Simple splitting by character

In [3]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./data/be-good.txt")

loaded_data = loader.load()

In [4]:
loaded_data

[Document(metadata={'source': './data/be-good.txt'}, page_content='Be good\n\nApril 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we started Y Combinator we came up with the\nphrase that became our motto: Make something people want.  We\'ve\nlearned a lot since then, but if I were choosing now that\'s still\nthe one I\'d pick.Another thing we tell founders is not to worry too much about the\nbusiness model, at least at first.  Not because making money is\nunimportant, but because it\'s so much easier than building something\ngreat.A couple weeks ago I realized that if you put those two ideas\ntogether, you get something surprising.  Make something people want.\nDon\'t worry too much about making money.  What you\'ve got is a\ndescription of a charity.When you get an unexpected result like this, it could either be a\nbug or a new discovery.  Either businesses aren\'t supposed to be\nlike charities, and we\'ve proven by reductio ad absurdum that o

In [5]:
loaded_data[0].page_content

'Be good\n\nApril 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we started Y Combinator we came up with the\nphrase that became our motto: Make something people want.  We\'ve\nlearned a lot since then, but if I were choosing now that\'s still\nthe one I\'d pick.Another thing we tell founders is not to worry too much about the\nbusiness model, at least at first.  Not because making money is\nunimportant, but because it\'s so much easier than building something\ngreat.A couple weeks ago I realized that if you put those two ideas\ntogether, you get something surprising.  Make something people want.\nDon\'t worry too much about making money.  What you\'ve got is a\ndescription of a charity.When you get an unexpected result like this, it could either be a\nbug or a new discovery.  Either businesses aren\'t supposed to be\nlike charities, and we\'ve proven by reductio ad absurdum that one\nor both of the principles we began with is false.  Or we have 

In [6]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [7]:
texts = text_splitter.create_documents([loaded_data[0].page_content])

In [8]:
len(texts)

2

In [9]:
texts[0]

Document(page_content='Be good')

In [11]:
texts[1]

Document(page_content='April 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we started Y Combinator we came up with the\nphrase that became our motto: Make something people want.  We\'ve\nlearned a lot since then, but if I were choosing now that\'s still\nthe one I\'d pick.Another thing we tell founders is not to worry too much about the\nbusiness model, at least at first.  Not because making money is\nunimportant, but because it\'s so much easier than building something\ngreat.A couple weeks ago I realized that if you put those two ideas\ntogether, you get something surprising.  Make something people want.\nDon\'t worry too much about making money.  What you\'ve got is a\ndescription of a charity.When you get an unexpected result like this, it could either be a\nbug or a new discovery.  Either businesses aren\'t supposed to be\nlike charities, and we\'ve proven by reductio ad absurdum that one\nor both of the principles we began with is false.  

#### Splitting with metadata

In [55]:
metadatas = [{"chunk": 0}, {"chunk": 1}]

documents = text_splitter.create_documents(
    [loaded_data[0].page_content, loaded_data[0].page_content], 
    metadatas=metadatas
)

In [58]:
documents[0]

Document(page_content='Be good', metadata={'chunk': 0})

In [59]:
print(documents[0])

page_content='Be good' metadata={'chunk': 0}


## Embeddings: transform chunks of text in chunks of numbers
* Vector databases work very fast with numbers.
* See the documentation page [here](https://python.langchain.com/v0.1/docs/modules/data_connection/text_embedding/).
* See the list of embedding model providers [here](https://python.langchain.com/v0.1/docs/integrations/text_embedding/).
* **The base Embeddings class in LangChain provides two methods: one for embedding documents and one for embedding a query**.

In [12]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [13]:
chunks_of_text =     [
        "Hi there!",
        "Hello!",
        "What's your name?",
        "Bond, James Bond",
        "Hello Bond!"
    ]

In [14]:
embeddings = embeddings_model.embed_documents(chunks_of_text)

In [17]:
# embeddings

In [18]:
len(embeddings)

5

In [19]:
len(embeddings[0])

1536

In [20]:
print(embeddings[0][:5])

[-0.020350975915789604, -0.007109511643648148, -0.022864697501063347, -0.026229018345475197, -0.0374772809445858]


In [21]:
embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")

In [22]:
len(embedded_query)

1536

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 003-embeddings.py
* In terminal, make sure you are in the directory of the file and run:
    * python 003-embeddings.py